# DCGAN 
###  this network is the same architecture as gans but with some enhancements and changes so I followed this architecture 

![DCGAN](images/DCGAN.png)

# import 

In [3]:
import tensorflow as tf 
import os 
from tensorflow.keras.layers import Dense , Conv2D , Conv2DTranspose , Flatten , Reshape  , BatchNormalization , Activation, Flatten ,LeakyReLU
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import mnist 
import numpy as np 
import matplotlib.pyplot as plt 
import  math

# generator 

In [6]:
class Generator(tf.keras.Model) : 
    def __init__(self  ) : 
        super().__init__() 
        self.dense_1 = Dense(7*7*128 ) 
        self.reshape = Reshape((7,7,128))

    
        self.sequential = Sequential([
             BatchNormalization() ,
             Activation('relu') , 
             Conv2DTranspose(128 , 5 , strides = 2 , padding = 'same' , use_bias=False) , 
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(64 , 5 , strides = 2  , padding = 'same' , use_bias=False) ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(32 , 5   , padding = 'same' , use_bias=False) ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(1 , 5  , padding = 'same' , use_bias=False) , 
             Activation('sigmoid')
        ])
    def call(self,  X ) : 
        X = self.reshape(self.dense_1(X))
        X = self.sequential(X)
        return  X 

# Discriminator 

In [7]:
class Discriminator(tf.keras.Model) : 
    def __init__(self ) : 
        super().__init__() 
        self.sequential = Sequential([
            LeakyReLU(.2), 
            Conv2D(32 , 5 , padding ='same' , strides = 2) , 
            LeakyReLU(.2),
            Conv2D(64 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(128 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(256 , 5 , padding ='same'  ) ,
            Flatten() , 
            Dense(1) , 
            Activation('sigmoid') 
        ])
    def call(self, X) : 
        return self.sequential(X)

# training loop 

In [27]:
def train_step(images , generator , discriminator  , loss_fn , gen_optimizer , dis_optimizer):
    noise = tf.random.normal([64, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = loss_fn(fake_output , tf.ones_like(fake_output))
        disc_real_loss = loss_fn(real_output, tf.ones_like(real_output)) 
        disc_fake_loss = loss_fn(fake_output , tf.zeros_like(fake_output))
        disc_loss = disc_real_loss + disc_fake_loss
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    gen_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    dis_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [28]:
def plot_images(generator,
                noise_input,
                show=False,
                step=0,
                model_name="gan"):
    
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict(noise_input)
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

# build the network 

In [29]:
def build_and_train_models(epochs):
    # load MNIST dataset
    (x_train, _), (_, _) = mnist.load_data()

    # reshape data for CNN as (28, 28, 1) and normalize
    x_train = np.reshape(x_train, [-1, 28, 28, 1])
    x_train = x_train.astype('float32') / 255

    ds = tf.data.Dataset.from_tensor_slices(x_train ) 
    ds = ds.shuffle(60000).batch(64)


    cross_entropy = tf.keras.losses.BinaryCrossentropy()
    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

    # build discriminator model
    discriminator = Discriminator()
    # build generator model
    generator = Generator()
    
    for epoch in range(epochs) : 
        for batch in ds : 
            train_step(images = batch , generator = generator  , discriminator = discriminator
            , loss_fn =cross_entropy , gen_optimizer = generator_optimizer , dis_optimizer =discriminator_optimizer )

        plot_images(generator ,tf.random.normal([16, 100]) , step = epoch  )


In [30]:
build_and_train_models(2)

1/1 [==============================] - 0s 104ms/step


2022-08-15 21:01:22.131096: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 8ms/step
